In [3]:
# ===================================================================
# TIMBER MOUNTAIN AI CHATBOT - DATA PROCESSING PIPELINE
# ===================================================================
# Step 1: Process and Combine Your Data
# This notebook processes A/B test metadata and PDF presentations
# to create a unified dataset for the Neo4j GraphRAG system.

import pandas as pd
import pdfplumber
import json
import os
from pathlib import Path
from datetime import datetime

print("🌲 Timber Mountain AI Chatbot - Data Processing Pipeline")
print("=" * 60)
print("✅ All libraries imported successfully!")
print(f"📅 Processing started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

🌲 Timber Mountain AI Chatbot - Data Processing Pipeline
✅ All libraries imported successfully!
📅 Processing started at: 2025-07-01 06:06:47


In [4]:
# ===================================================================
# STEP 1: LOAD AND ANALYZE A/B TEST METADATA
# ===================================================================

# Define file paths
project_root = Path("..")
metadata_path = project_root / "2 - Synthetic Metadata" / "Timber Mountain - AB Test Metadata.xlsx"
pdf_directory = project_root / "3 - Synthetic A:B Test Results Decks" / "2 - Results Presentations"

print("📊 LOADING A/B TEST METADATA")
print("-" * 40)

try:
    # Load metadata from Excel file
    metadata_df = pd.read_excel(metadata_path, sheet_name='Metadata')
    
    print(f"✅ Successfully loaded metadata for {len(metadata_df)} A/B tests")
    print(f"📁 Metadata file: {metadata_path.name}")
    print(f"📋 Columns: {list(metadata_df.columns)}")
    
    # Display summary of tests
    print(f"\n🧪 A/B TEST OVERVIEW:")
    print("-" * 40)
    for i, row in metadata_df.iterrows():
        print(f"{i+1}. {row['Test Name']}")
        print(f"   📄 PDF: {row['PDF File Name']}")
        print(f"   📅 Duration: {row['Test Launch'].strftime('%Y-%m-%d')} → {row['Test End'].strftime('%Y-%m-%d')}")
        print(f"   🎯 Target: {row['Target Segment']}")
        print()
    
    # Create metadata lookup dictionary for efficient access
    metadata_lookup = {}
    for i, row in metadata_df.iterrows():
        pdf_filename = row['PDF File Name']
        metadata_lookup[pdf_filename] = {
            'test_name': row['Test Name'],
            'test_launch': row['Test Launch'].strftime('%Y-%m-%d'),
            'test_end': row['Test End'].strftime('%Y-%m-%d'),
            'country': row['Country'],
            'target_segment': row['Target Segment'],
            'page_placement': row['Page / Placement'],
            'test_hypothesis': row['Test Hypothesis'],
            'test_result': row['Test Result & Interpretation']
        }
    
    print(f"🔍 Created metadata lookup for {len(metadata_lookup)} tests")
    
except FileNotFoundError:
    print(f"❌ ERROR: Metadata file not found at {metadata_path}")
    raise
except Exception as e:
    print(f"❌ ERROR loading metadata: {e}")
    raise

📊 LOADING A/B TEST METADATA
----------------------------------------
✅ Successfully loaded metadata for 5 A/B tests
📁 Metadata file: Timber Mountain - AB Test Metadata.xlsx
📋 Columns: ['Test Name', 'PDF File Name', 'Test Launch', 'Test End', 'Country', 'Target Segment', 'Page / Placement', 'Test Hypothesis', 'Test Result & Interpretation']

🧪 A/B TEST OVERVIEW:
----------------------------------------
1. Homepage: Domestic vs. International Visitors — Content Personalization Test
   📄 PDF: 1 - Locale-Aware-Experience-How-We-Boosted-International-Conversions-at-Timber-Mountain.pdf
   📅 Duration: 2024-07-08 → 2024-07-28
   🎯 Target: Browser-locale ≠ “en-US” (Int’l) vs “en-US” (Domestic)

2. AI Planner: Add Verified Star Ratings — Trust & Adoption Test
   📄 PDF: 2 - Wild-Willy-AI-Planner-Trust-and-Adoption-AB-Test-Results.pdf
   📅 Duration: 2024-08-05 → 2024-08-25
   🎯 Target: Wild Willy AI Travel Planner Users

3. Checkout: Unified Booking.com Bundle Flow — Seamless-Booking Test
   📄 PDF

In [5]:
# ===================================================================
# STEP 2: PROCESS PDF FILES AND EXTRACT TEXT CONTENT
# ===================================================================

print("\n📄 PROCESSING PDF FILES")
print("-" * 40)

# Find all PDF files in the results presentations directory
pdf_files = [f for f in pdf_directory.iterdir() if f.suffix.lower() == '.pdf']

if not pdf_files:
    print(f"❌ ERROR: No PDF files found in {pdf_directory}")
    raise FileNotFoundError("PDF files not found")

print(f"📂 Found {len(pdf_files)} PDF files to process:")
for pdf_file in sorted(pdf_files):
    print(f"   - {pdf_file.name}")

# Dictionary to store extracted text content
pdf_text_content = {}

print(f"\n🔍 EXTRACTING TEXT FROM PDF FILES:")
print("-" * 40)

for pdf_path in sorted(pdf_files):
    print(f"Processing: {pdf_path.name}")
    
    try:
        # Extract all text from the PDF
        with pdfplumber.open(pdf_path) as pdf:
            # Extract text from all pages
            all_pages_text = []
            
            for page_num, page in enumerate(pdf.pages, 1):
                page_text = page.extract_text()
                if page_text:
                    all_pages_text.append(f"=== PAGE {page_num} ===\n{page_text}")
                else:
                    all_pages_text.append(f"=== PAGE {page_num} ===\n[No text extracted]")
            
            # Combine all pages
            full_text = "\n\n".join(all_pages_text)
            
            # Store the text content
            pdf_text_content[pdf_path.name] = {
                'file_path': str(pdf_path),
                'page_count': len(pdf.pages),
                'text_length': len(full_text),
                'full_text': full_text
            }
            
            print(f"   ✅ Extracted {len(full_text):,} characters from {len(pdf.pages)} pages")
    
    except Exception as e:
        print(f"   ❌ ERROR processing {pdf_path.name}: {e}")
        pdf_text_content[pdf_path.name] = {
            'file_path': str(pdf_path),
            'page_count': 0,
            'text_length': 0,
            'full_text': f"[ERROR: Could not extract text - {e}]",
            'error': str(e)
        }

print(f"\n📊 PDF PROCESSING SUMMARY:")
print(f"   • Successfully processed: {len([k for k, v in pdf_text_content.items() if 'error' not in v])}")
print(f"   • Failed to process: {len([k for k, v in pdf_text_content.items() if 'error' in v])}")
print(f"   • Total text extracted: {sum(v['text_length'] for v in pdf_text_content.values()):,} characters")


📄 PROCESSING PDF FILES
----------------------------------------
📂 Found 5 PDF files to process:
   - 1 - Locale-Aware-Experience-How-We-Boosted-International-Conversions-at-Timber-Mountain.pdf
   - 2 - Wild-Willy-AI-Planner-Trust-and-Adoption-AB-Test-Results.pdf
   - 3 - Timber-Mountain-Unified-Bundle-Flow-Checkout-Test-Results.pdf
   - 4 - Timber-Mountain-CTA-Copy-Test-Results.pdf
   - 5 - Homepage-Special-Offers-Carousel-Merchandising-Test-Results.pdf

🔍 EXTRACTING TEXT FROM PDF FILES:
----------------------------------------
Processing: 1 - Locale-Aware-Experience-How-We-Boosted-International-Conversions-at-Timber-Mountain.pdf
   ✅ Extracted 2,992 characters from 7 pages
Processing: 2 - Wild-Willy-AI-Planner-Trust-and-Adoption-AB-Test-Results.pdf
   ✅ Extracted 4,031 characters from 9 pages
Processing: 3 - Timber-Mountain-Unified-Bundle-Flow-Checkout-Test-Results.pdf
   ✅ Extracted 4,595 characters from 10 pages
Processing: 4 - Timber-Mountain-CTA-Copy-Test-Results.pdf
   ✅ Extract

In [7]:
# ===================================================================
# STEP 3: COMBINE METADATA WITH PDF TEXT CONTENT
# ===================================================================

print("\n🔗 COMBINING METADATA WITH PDF CONTENT")
print("-" * 40)

unified_documents = []
processing_summary = {
    'successful_matches': 0,
    'missing_metadata': 0,
    'missing_pdf_content': 0,
    'total_processed': 0
}

# Iterate through each PDF file and combine with metadata
for pdf_filename in pdf_text_content.keys():
    print(f"Combining data for: {pdf_filename}")
    
    # Get metadata for this PDF
    metadata = metadata_lookup.get(pdf_filename, {})
    
    # Get PDF text content
    pdf_data = pdf_text_content.get(pdf_filename, {})
    
    # Create unified document
    unified_doc = {
        'document_id': f"timber_mountain_{len(unified_documents) + 1:03d}",
        'source_pdf_filename': pdf_filename,
        'source_pdf_path': pdf_data.get('file_path', ''),
        'pdf_processing': {
            'page_count': pdf_data.get('page_count', 0),
            'text_length': pdf_data.get('text_length', 0),
            'extraction_error': pdf_data.get('error', None)
        },
        'metadata': metadata,
        'content': {
            'full_text': pdf_data.get('full_text', ''),
            'processed_timestamp': datetime.now().isoformat()
        }
    }
    
    # Update processing summary
    processing_summary['total_processed'] += 1
    
    if metadata:
        processing_summary['successful_matches'] += 1
        print(f"   ✅ Metadata found: {metadata.get('test_name', 'Unknown')}")
    else:
        processing_summary['missing_metadata'] += 1
        print(f"   ⚠️  No metadata found for {pdf_filename}")
    
    if pdf_data.get('full_text') and 'error' not in pdf_data:
        print(f"   ✅ PDF content: {pdf_data['text_length']:,} characters")
    else:
        processing_summary['missing_pdf_content'] += 1
        print(f"   ⚠️  PDF content extraction failed")
    
    unified_documents.append(unified_doc)
    print()

print(f"📊 COMBINATION SUMMARY:")
print(f"   • Total documents processed: {processing_summary['total_processed']}")
print(f"   • Successful metadata matches: {processing_summary['successful_matches']}")
print(f"   • Missing metadata: {processing_summary['missing_metadata']}")
print(f"   • PDF extraction failures: {processing_summary['missing_pdf_content']}")
print(f"   • Unified documents created: {len(unified_documents)}")

# Display sample unified document structure
if unified_documents:
    print(f"\n📋 SAMPLE UNIFIED DOCUMENT STRUCTURE:")
    print("-" * 40)
    sample_doc = unified_documents[0]
    print(f"Document ID: {sample_doc['document_id']}")
    print(f"Source PDF: {sample_doc['source_pdf_filename']}")
    print(f"Test Name: {sample_doc['metadata'].get('test_name', 'N/A')}")
    print(f"Content Length: {sample_doc['content']['full_text'][:100]}...")
    print(f"Metadata Keys: {list(sample_doc['metadata'].keys()) if sample_doc['metadata'] else 'None'}")


🔗 COMBINING METADATA WITH PDF CONTENT
----------------------------------------
Combining data for: 1 - Locale-Aware-Experience-How-We-Boosted-International-Conversions-at-Timber-Mountain.pdf
   ✅ Metadata found: Homepage: Domestic vs. International Visitors — Content Personalization Test
   ✅ PDF content: 2,992 characters

Combining data for: 2 - Wild-Willy-AI-Planner-Trust-and-Adoption-AB-Test-Results.pdf
   ✅ Metadata found: AI Planner: Add Verified Star Ratings — Trust & Adoption Test
   ✅ PDF content: 4,031 characters

Combining data for: 3 - Timber-Mountain-Unified-Bundle-Flow-Checkout-Test-Results.pdf
   ✅ Metadata found: Checkout: Unified Booking.com Bundle Flow — Seamless-Booking Test
   ✅ PDF content: 4,595 characters

Combining data for: 4 - Timber-Mountain-CTA-Copy-Test-Results.pdf
   ✅ Metadata found: Site-wide CTA Copy: “Learn More” vs. “Explore More” — Engagement Nudge Test
   ✅ PDF content: 4,244 characters

Combining data for: 5 - Homepage-Special-Offers-Carousel-Merch

In [8]:
# ===================================================================
# STEP 4: GENERATE UNIFIED JSON OUTPUT FOR NEO4J GRAPHRAG
# ===================================================================

print("\n💾 GENERATING UNIFIED JSON OUTPUT")
print("-" * 40)

# Define output path
output_json_path = project_root / "processed_documents.json"

# Create the final output structure optimized for GraphRAG
final_output = {
    'processing_metadata': {
        'created_timestamp': datetime.now().isoformat(),
        'source_excel_file': str(metadata_path),
        'source_pdf_directory': str(pdf_directory),
        'total_documents': len(unified_documents),
        'processing_summary': processing_summary
    },
    'documents': unified_documents
}

try:
    # Save to JSON file with proper formatting
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(final_output, f, indent=2, ensure_ascii=False)
    
    # Calculate file size
    file_size_mb = output_json_path.stat().st_size / (1024 * 1024)
    
    print(f"✅ Successfully saved unified dataset!")
    print(f"📁 Output file: {output_json_path.name}")
    print(f"📏 File size: {file_size_mb:.2f} MB")
    print(f"📊 Contains {len(unified_documents)} documents")
    
    # Display JSON structure summary
    print(f"\n📋 JSON STRUCTURE SUMMARY:")
    print("-" * 40)
    print("└── processing_metadata")
    print("    ├── created_timestamp")
    print("    ├── source_excel_file") 
    print("    ├── source_pdf_directory")
    print("    ├── total_documents")
    print("    └── processing_summary")
    print("└── documents (array)")
    print("    └── [document]")
    print("        ├── document_id")
    print("        ├── source_pdf_filename")
    print("        ├── source_pdf_path")
    print("        ├── pdf_processing")
    print("        ├── metadata")
    print("        └── content")
    
    print(f"\n🎯 READY FOR NEXT STEP: Populating Neo4j Graph Database")
    print(f"📝 Use '{output_json_path.name}' as input for graph population")
    
except Exception as e:
    print(f"❌ ERROR saving JSON file: {e}")
    raise

# Display sample of final JSON structure
print(f"\n📄 SAMPLE JSON OUTPUT:")
print("-" * 40)
sample_output = {
    'processing_metadata': final_output['processing_metadata'],
    'documents': [final_output['documents'][0]] if final_output['documents'] else []
}

print(json.dumps(sample_output, indent=2)[:1000] + "..." if len(str(sample_output)) > 1000 else json.dumps(sample_output, indent=2))


💾 GENERATING UNIFIED JSON OUTPUT
----------------------------------------
✅ Successfully saved unified dataset!
📁 Output file: processed_documents.json
📏 File size: 0.03 MB
📊 Contains 5 documents

📋 JSON STRUCTURE SUMMARY:
----------------------------------------
└── processing_metadata
    ├── created_timestamp
    ├── source_excel_file
    ├── source_pdf_directory
    ├── total_documents
    └── processing_summary
└── documents (array)
    └── [document]
        ├── document_id
        ├── source_pdf_filename
        ├── source_pdf_path
        ├── pdf_processing
        ├── metadata
        └── content

🎯 READY FOR NEXT STEP: Populating Neo4j Graph Database
📝 Use 'processed_documents.json' as input for graph population

📄 SAMPLE JSON OUTPUT:
----------------------------------------
{
  "processing_metadata": {
    "created_timestamp": "2025-07-01T06:16:03.658141",
    "source_excel_file": "../2 - Synthetic Metadata/Timber Mountain - AB Test Metadata.xlsx",
    "source_pdf_directory

In [9]:
# ===================================================================
# STEP 5: DATA VALIDATION AND SUMMARY STATISTICS
# ===================================================================

print("\n✅ DATA VALIDATION AND FINAL SUMMARY")
print("=" * 60)

# Validation checks
validation_results = {
    'all_tests_processed': len(unified_documents) == len(metadata_df),
    'all_metadata_matched': processing_summary['missing_metadata'] == 0,
    'all_pdfs_extracted': processing_summary['missing_pdf_content'] == 0,
    'output_file_created': output_json_path.exists(),
    'total_characters': sum(len(doc['content']['full_text']) for doc in unified_documents),
    'avg_characters_per_doc': 0
}

if len(unified_documents) > 0:
    validation_results['avg_characters_per_doc'] = validation_results['total_characters'] / len(unified_documents)

# Display validation results
print("🔍 VALIDATION RESULTS:")
print("-" * 30)
print(f"✅ All A/B tests processed: {validation_results['all_tests_processed']} ({len(unified_documents)}/{len(metadata_df)})")
print(f"✅ All metadata matched: {validation_results['all_metadata_matched']} ({processing_summary['successful_matches']}/{len(unified_documents)})")
print(f"✅ All PDFs extracted: {validation_results['all_pdfs_extracted']} (failures: {processing_summary['missing_pdf_content']})")
print(f"✅ Output file created: {validation_results['output_file_created']} ({output_json_path.name})")

print(f"\n📊 CONTENT STATISTICS:")
print("-" * 30)
print(f"Total text content: {validation_results['total_characters']:,} characters")
print(f"Average per document: {validation_results['avg_characters_per_doc']:,.0f} characters")

# Document-level statistics
print(f"\n📋 DOCUMENT-LEVEL BREAKDOWN:")
print("-" * 30)
for i, doc in enumerate(unified_documents, 1):
    test_name = doc['metadata'].get('test_name', 'Unknown Test')
    char_count = len(doc['content']['full_text'])
    page_count = doc['pdf_processing']['page_count']
    
    print(f"{i}. {test_name[:50]}{'...' if len(test_name) > 50 else ''}")
    print(f"   📄 {page_count} pages, {char_count:,} characters")

# Check for potential issues
print(f"\n⚠️  POTENTIAL ISSUES:")
print("-" * 30)
issues_found = 0

for doc in unified_documents:
    if doc['pdf_processing'].get('extraction_error'):
        print(f"❌ PDF extraction error in: {doc['source_pdf_filename']}")
        issues_found += 1
    
    if not doc['metadata']:
        print(f"⚠️  Missing metadata for: {doc['source_pdf_filename']}")
        issues_found += 1
    
    if len(doc['content']['full_text']) < 100:
        print(f"⚠️  Very short content in: {doc['source_pdf_filename']} ({len(doc['content']['full_text'])} chars)")
        issues_found += 1

if issues_found == 0:
    print("✅ No issues detected - data quality looks good!")

# Final processing summary
print(f"\n🎯 PROCESSING COMPLETE!")
print("=" * 60)
print(f"📈 Successfully processed {len(unified_documents)} A/B test documents")
print(f"💾 Output saved to: {output_json_path}")
print(f"🔗 Ready for Neo4j graph database population")
print(f"📅 Processing completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# Next steps guidance
print(f"\n📋 NEXT STEPS:")
print("-" * 30)
print("1. 🗄️  Set up Neo4j database connection")
print("2. 🏗️  Create graph schema for A/B test data")
print("3. 📥 Import processed_documents.json into Neo4j")
print("4. 🔍 Build GraphRAG query system")
print("5. 🤖 Integrate with LangChain for chatbot responses")
print(f"6. 🌐 Deploy Streamlit frontend")

print(f"\n🌲 Timber Mountain AI Chatbot data processing pipeline complete! 🌲")


✅ DATA VALIDATION AND FINAL SUMMARY
🔍 VALIDATION RESULTS:
------------------------------
✅ All A/B tests processed: True (5/5)
✅ All metadata matched: True (5/5)
✅ All PDFs extracted: True (failures: 0)
✅ Output file created: True (processed_documents.json)

📊 CONTENT STATISTICS:
------------------------------
Total text content: 21,031 characters
Average per document: 4,206 characters

📋 DOCUMENT-LEVEL BREAKDOWN:
------------------------------
1. Homepage: Domestic vs. International Visitors — Co...
   📄 7 pages, 2,992 characters
2. AI Planner: Add Verified Star Ratings — Trust & Ad...
   📄 9 pages, 4,031 characters
3. Checkout: Unified Booking.com Bundle Flow — Seamle...
   📄 10 pages, 4,595 characters
4. Site-wide CTA Copy: “Learn More” vs. “Explore More...
   📄 10 pages, 4,244 characters
5. Homepage: Special Offers Carousel — Merchandising ...
   📄 10 pages, 5,169 characters

⚠️  POTENTIAL ISSUES:
------------------------------
✅ No issues detected - data quality looks good!

🎯 PR